# CPSC 368 SQL Create Views (KNM Neighbours)

## Prerequisites

- All of the code below is to be run AFTER running `cpsc_368_project_knm_csv_to_sql.ipynb` for creating the file "knm_datasetup.sql".

To upload tables: 

- Import "knm_datasetup.sql" to SSH server: `scp "<PATH_TO_START>/knm_datasetup.sql" <CWL>@remote.students.cs.ubc.ca:/<PATH_TO_END>/`
- SSH connect to UBC department servers: `ssh <CWL>@remote.students.cs.ubc.ca`
- Move to location of "knm_datasetup.sql": `cd <PATH_TO_SQL>`
- Enter MySQL within SSH: `rlwrap sqlplus ora_<CWL>@stu`
    - Password: `<studentnum>`
- In MySQL, run `start knm_datasetup.sql;` to run all SQL imports

To run SQL in notebook (only tested within Jupyter Lab):

- Open terminal
- Run `ssh -l <CWL> -L 127.0.0.1:1522:dbhost.students.cs.ubc.ca:1522 remote.students.cs.ubc.ca` to access SSH server

Reference for connection to database: https://www.students.cs.ubc.ca/~cs-368/resources/connecting-to-db.html

## Loading Data and Packages

In [1]:
import oracledb
import pandas as pd

In [2]:
# INPUT USER INFO HERE
CWL = ""
studentnum = ""

## Test Run

In [3]:
# Simple test run
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    for row in cur.execute(
        """
        SELECT * FROM KFF2019_adult FETCH FIRST 3 ROWS ONLY
        """
        ):
        print(row)
    print("=========================")
    
    for row in cur.execute(
        """
        SELECT * FROM KFF2019_female FETCH FIRST 3 ROWS ONLY
        """
        ):
        print(row)
    print("=========================")

    for row in cur.execute(
        """
        SELECT * FROM KFF2019_male FETCH FIRST 3 ROWS ONLY
        """
        ):
        print(row)
    print("=========================")

    for row in cur.execute(
        """
        SELECT * FROM USCDI_filter FETCH FIRST 3 ROWS ONLY
        """
        ):
        print(row)
    print("=========================")
except Exception as e:
    print(f"Error executing SQL query: {e}")
else: 
    print("System runs without error, please proceed.")
finally: 
    cur.close()
    connection.close()

('United States', 0.612, 0.077, 0.145, 0.02, 0.016, 0.129, 1.0)
('Alabama', 0.598, 0.075, 0.116, 0.036, 0.025, 0.149, 1.0)
('Alaska', 0.582, 0.047, 0.159, 0.011, 0.047, 0.153, 1.0)
('United States', 0.607, 0.079, 0.165, 0.02, 0.015, 0.114, 1.0)
('Alabama', 0.588, 0.078, 0.139, 0.038, 0.024, 0.133, 1.0)
('Alaska', 0.591, 0.041, 0.17, 0.013, 0.065, 0.119, 1.0)
('United States', 0.618, 0.076, 0.124, 0.02, 0.017, 0.145, 1.0)
('Alabama', 0.609, 0.071, 0.091, 0.035, 0.027, 0.167, 1.0)
('Alaska', 0.573, 0.053, 0.148, 0.01, 0.028, 0.187, 1.0)
(2015, 2019, 'Oklahoma', 'Cancer', 'Breast cancer mortality among all females, underlying cause', 'per 100,000', 'Age-adjusted Rate', 22.5, 'Overall', 'Overall')
(2015, 2019, 'Alaska', 'Cancer', 'Cervical cancer mortality among all females, underlying cause', 'per 100,000', 'Age-adjusted Rate', 1.7, 'Sex', 'Female')
(2015, 2019, 'Iowa', 'Cancer', 'Breast cancer mortality among all females, underlying cause', 'per 100,000', 'Age-adjusted Rate', 18.4, 'Over

## `KFF2019_NEW`

- There are 3 KFF datasets: one for all adults aged 19-64, and two for males and females aged 19-64.
- Each dataset has a corresponding `Group` column applied to them before they are joined on `Location`.
- Since our focus is on uninsured adults exclusively, only the `Uninsured` column of values are acquired for each individual dataset, which are then grouped by location to create the columns `All_Uninsured`, `Female_Uninsured`, and `Male_Uninsured`, corresponding to the proportion of uninsured individuals in each category for each country.

| Column | Description | Data Type | Property |
| ------- | ------- | ------- | ------- |
| `Location`  | State within U.S. | `VARCHAR2(50)` | `PRIMARY KEY` |
| `All_Uninsured` | Proportion of uninsured individuals aged between 19 and 64 | `DECIMAL(19, 18)` | N/A |
| `Female_Uninsured` | Proportion of uninsured female individuals aged between 19 and 64 | `DECIMAL(19, 18)` | N/A |
| `Male_Uninsured` | Proportion of uninsured male individuals aged between 19 and 64 | `DECIMAL(19, 18)` | N/A |

In [4]:
# KFF2019_NEW
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    # Remove KFF2019_NEW view only if it exists
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'KFF2019_NEW'")
    view_exists = cur.fetchone()[0]
    if view_exists:
        cur.execute("DROP VIEW KFF2019_NEW CASCADE CONSTRAINTS")
    
    # Create KFF2019_NEW view
    cur.execute(
        """
        CREATE VIEW KFF2019_NEW AS 
        SELECT kffa1."Location" AS Location, kffa1."Uninsured" AS All_Uninsured, kfff1."Uninsured" AS Female_Uninsured, kffm1."Uninsured" AS Male_Uninsured
        FROM KFF2019_adult kffa1
        INNER JOIN KFF2019_female kfff1 ON kffa1."Location" = kfff1."Location"
        INNER JOIN KFF2019_male kffm1 ON kffa1."Location" = kffm1."Location"
        WHERE kffa1."Location" != 'United States'
        """
    )
    cur.execute(
        """
        SELECT * 
        FROM KFF2019_NEW
        """
    )
    
    # Fetch SELECT results 
    rows = cur.fetchall()
    
    # Get column names from cursor
    columns = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=columns)
    
    print("Completed dataframe.")
    
except Exception as e:
    print(f"Error executing SQL query: {e}")

finally:
    # Close cursor
    cur.close()
    connection.close()

Error executing SQL query: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/


In [5]:
display(df.head())

NameError: name 'df' is not defined

## `USCDI`

- The column `Has2019` is created to determine if the value is relevant to our questions
- `Range` is created to assist in providing the average data value `AvgDataValue` across the range of years, given that some values are obtained for a range greater than 1 year. 
    - `USCDI_MID` is created within SQL script to ensure `Range` could be calculated.

| Column | Description | Data Type | Property |
| ------- | ------- | ------- | ------- |
| `YearStart`  | Start year of measurements | `NUMBER(4, 0)` | `PRIMARY KEY` |
| `YearEnd` | End year of measurements | `NUMBER(4, 0)` | `PRIMARY KEY` |
| `LocationDesc` | State within U.S. | `VARCHAR2(50)` | `PRIMARY KEY` |
| `Topic` | Topic of interest | `VARCHAR2(30)` | `PRIMARY KEY` |
| `Question`  | Question of interest, based on `Topic` | `VARCHAR2(100)` | `PRIMARY KEY` |
| `DataValueUnit` | Unit of data value depending on `Topic` and `Question` | `VARCHAR2(20)` | `PRIMARY KEY` |
| `DataValueType` | Type of data value (e.g. Crude value, age-adjusted) | `VARCHAR2(20)` | `PRIMARY KEY` |
| `DataValue` | Data value, with specific interpretation dependent on its `DataValueType`, `DataValueUnit`, `Topic` and `Question` | `DECIMAL(24, 18)` | N/A |
| `StratificationCategory1` | Category to stratify data; includes "Age", "Sex", "Race/Ethnicity" and "Overall" | `VARCHAR(10)` | `PRIMARY KEY` |
| `Stratification1` | Specific group within `StratificationCategory1` | `VARCHAR(10)` | `PRIMARY KEY` |
| `Has2019` | Boolean on whether or not 2019 is in the data | `NUMBER(1,0)` | `NOT NULL` |
| `Range` | Number of years between `YearStart` and `YearEnd` | `NUMBER(2,0)` | `NOT NULL` |
| `AvgDataValue`  | $\frac{DataValue}{Range}$ | `DECIMAL(24, 18)` | N/A |

In [ ]:
# USCDI
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    # Remove USCDI view only if it exists
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'USCDI'")
    view_exists = cur.fetchone()[0]
    if view_exists:
        cur.execute("DROP VIEW USCDI CASCADE CONSTRAINTS")
    
    # Create USCDI view
    cur.execute(
        """
        CREATE VIEW USCDI AS 
        SELECT USCDI_MID."YearStart" AS YearStart, 
            USCDI_MID."YearEnd" AS YearEnd, 
            USCDI_MID."LocationDesc" AS LocationDesc, 
            USCDI_MID."Topic" AS Topic, 
            USCDI_MID."Question" AS Question, 
            USCDI_MID."DataValueUnit" AS DataValueUnit, 
            USCDI_MID."DataValueType" AS DataValueType, 
            USCDI_MID."DataValue" AS DataValue, 
            USCDI_MID."StratificationCategory1" AS StratificationCategory1, 
            USCDI_MID."Stratification1" AS Stratification1, 
            USCDI_MID."Has2019" AS Has2019, 
            USCDI_MID."Range" AS Range, 
            (USCDI_MID."DataValue" / USCDI_MID."Range") AS AvgDataValue
        FROM (
            SELECT cdif1."YearStart", cdif1."YearEnd", cdif1."LocationDesc", cdif1."Topic", cdif1."Question", 
                   cdif1."DataValueUnit", cdif1."DataValueType", cdif1."DataValue", 
                   cdif1."StratificationCategory1", cdif1."Stratification1", 
                   CAST(
                       CASE 
                           WHEN ((cdif1."YearStart" <= 2019) AND (cdif1."YearEnd" >= 2019)) THEN 1
                           ELSE 0
                       END AS NUMBER(1, 0)
                   ) AS "Has2019", 
                   CAST(
                       (cdif1."YearEnd" - cdif1."YearStart" + 1) AS NUMBER(2, 0)
                   ) AS "Range"
            FROM USCDI_filter cdif1
            WHERE cdif1."LocationDesc" != 'United States'
        ) USCDI_MID
        """
    )
    cur.execute(
        """
        SELECT * 
        FROM USCDI
        """
    )
    
    # Fetch SELECT results 
    rows2 = cur.fetchall()
    
    # Get column names from cursor
    columns2 = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    df2 = pd.DataFrame(rows2, columns=columns2)
    
    print("Completed dataframe.")
    
except Exception as e:
    print(f"Error executing SQL query: {e}")

finally:
    # Close cursor and connection
    cur.close()
    connection.close()

In [ ]:
display(df2.head())